In [19]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [20]:
# evaluate multinomial logistic regression model
import numpy as np
import pandas as pd
from sklearn.model_selection import *
from sklearn.metrics import confusion_matrix

# from sklearn.model_selection import cross_val_score, cross_validate
# from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression

import plotly.express as px
from IPython.display import display
import statsmodels.api as sm

<IPython.core.display.Javascript object>

In [21]:
data = pd.read_csv("Data/data_clean.csv", index_col=[0])
display(data)

X = data.drop(["score", "elapse", "gender", "age"], axis=1)
Y = data["gender"]

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.3, random_state=50
)

,score,Influence,Modesty,Daring,Confidence,Ruling,Responsibility,IntrovExtro,Success,Humble,...,Confidence in Success,MakeBelieve,Leadership Origin,Biography,Public Opinion,Capability,Extraordinary,elapse,gender,age
0,18,2,2,2,2,1,2,1,2,2,...,1,1,2,2,2,1,2,211,1,50
1,6,2,2,2,1,2,2,1,2,1,...,2,1,2,2,2,2,1,149,1,40
2,27,1,2,2,1,2,1,2,1,2,...,1,2,1,1,2,1,2,168,1,28
3,29,1,1,2,2,2,1,2,1,1,...,1,2,1,2,2,1,1,230,1,37
4,6,1,2,1,1,1,2,1,2,1,...,2,1,2,2,2,0,1,389,1,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11238,1,2,2,2,1,1,2,1,2,1,...,2,1,2,2,2,2,1,243,1,24
11239,10,2,2,1,1,1,2,1,1,1,...,2,1,2,1,2,2,1,197,1,39
11240,6,1,2,2,1,1,2,1,2,1,...,2,1,2,2,2,2,1,447,2,33
11241,12,2,2,1,1,1,1,1,1,1,...,1,2,2,2,2,2,1,167,1,24


<IPython.core.display.Javascript object>

In [22]:
# Building the Multinomial Logistic Model with sklearn
model = LogisticRegression(multi_class="multinomial", solver="sag", fit_intercept=False)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=4, random_state=1)
n_scores = cross_validate(
    model, X_train, Y_train, scoring="accuracy", cv=cv, n_jobs=-1, return_estimator=True
)

print(
    "Mean Accuracy: %.3f (+/- %.3f)"
    % (np.mean(n_scores["test_score"]), np.std(n_scores["test_score"]))
)

# print(np.argmin(n_scores["test_score"]))

Mean Accuracy: 0.639 (+/- 0.017)


<IPython.core.display.Javascript object>

In [23]:
# for model in n_scores["estimator"]:
#     print(model.coef_)

# n_scores["estimator"].coef_

model.fit(X_train, Y_train)

# Create dataframe for coefficients
model_coef = pd.DataFrame(model.coef_.T)
model_coef.insert(0, "Questions", X.columns)
model_coef.columns = ["Questions", "Male", "Female", "Other"]
Y_pred = model.predict(X_test)

# Create dataframe to assess predictions
model_eval = pd.DataFrame({"Actual": Y_test, "Predicted": Y_pred})
display(model_eval)
display(model_coef)

,Actual,Predicted
591,1,1
2460,2,1
3615,2,2
2395,1,1
6474,1,1
...,...,...
3270,1,2
517,1,2
4199,2,2
2111,2,2


,Questions,Male,Female,Other
0,Influence,-0.153250,-0.197311,0.350561
1,Modesty,0.027702,0.238560,-0.266262
2,Daring,0.056751,0.052513,-0.109264
3,Confidence,0.103762,-0.056107,-0.047655
4,Ruling,0.490998,-0.009351,-0.481648
5,Responsibility,-0.066381,0.080208,-0.013827
6,IntrovExtro,0.032260,0.385583,-0.417843
7,Success,0.101536,-0.012112,-0.089424
8,Humble,0.297113,0.193004,-0.490117
9,Leadership,0.094729,0.140732,-0.235461


<IPython.core.display.Javascript object>

In [24]:
# Using Statsmodels.api
# logit_model = sm.MNLogit(Y, X)
# result = logit_model.fit()
# print(result.summary2())

<IPython.core.display.Javascript object>

In [26]:
conmat = confusion_matrix(Y_test, Y_pred)
display(conmat)
fig = px.imshow(
    np.round((conmat / conmat.sum()) * 100, 3),
    labels=dict(x="Predicted Values", y="True Values", color="Percentage Classified"),
    x=["Male", "Female", "Other"],
    y=["Male", "Female", "Other"],
)
fig.update_xaxes(side="top")
fig.show()

array([[1445,  482,    1],
       [ 727,  668,    0],
       [   9,    4,    0]], dtype=int64)

<IPython.core.display.Javascript object>